## Objectif est de reconstruire CT scans 3D à partir des .dcm du challenge RSNA

In [8]:
from monai.data import PersistentDataset, DataLoader
import pandas as pd
import numpy as np
from pathlib import Path
import monai.transforms as T
import torch
# Using MONAI for medical imaging tasks
# ---- Config ----
csv_path = Path("/home/tibia/Projet_Hemorragie/Seg_hemorragie/Classification/Classification_RSNA/data/csv/val_fold0.csv")
dicom_dir = Path("/home/tibia/Projet_Hemorragie/Seg_hemorragie/Classification/Classification_RSNA/data/rsna-intracranial-hemorrhage-detection/stage_2_train")
cache_dir = Path("./persistent_cache/fold0")  

label_cols = ['any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']
df = pd.read_csv(csv_path)

In [9]:
import pandas as pd
from pathlib import Path
import os
import shutil
import subprocess



output_nifti_dir = Path("/home/tibia/Projet_Hemorragie/Seg_hemorragie/Reconstructions_Nifti")
# Créer le répertoire de sortie s'il n'existe pas
output_nifti_dir.mkdir(parents=True, exist_ok=True)

# ======================
# FONCTION DE RECONSTRUCTION
# ======================
def reconstruct_3d_scans_from_dicom(csv_file: Path, dicom_root: Path, output_root: Path):
    """
    Lit un fichier CSV pour grouper les fichiers DICOM par PatientID et les convertit
    en volumes 3D NIfTI à l'aide de dcm2niix.

    Args:
        csv_file (Path): Chemin vers le fichier CSV de métadonnées.
        dicom_root (Path): Répertoire contenant tous les fichiers DICOM (DICOM_ID.dcm).
        output_root (Path): Répertoire où les fichiers NIfTI reconstruits seront sauvegardés.
    """
    print(">>> Démarrage de la reconstruction des scans 3D...")

    # Lire le fichier CSV
    try:
        df = pd.read_csv(csv_file)
    except FileNotFoundError:
        print(f"Erreur : Le fichier CSV {csv_file} n'a pas été trouvé.")
        return

    # S'assurer que le répertoire de sortie existe
    output_root.mkdir(parents=True, exist_ok=True)

    # Grouper les données par PatientID
    patient_groups = df.groupby('PatientID')

    for patient_id, group in patient_groups:
        print(f"Traitement du patient : {patient_id} avec {len(group)} images...")

        # Créer un répertoire temporaire pour ce patient
        temp_patient_dir = Path(output_root) / f"temp_{patient_id}"
        temp_patient_dir.mkdir(exist_ok=True)

        # Copier les fichiers DICOM correspondants dans le répertoire temporaire
        for filename in group['filename']:
            source_file = dicom_root / filename
            destination_file = temp_patient_dir / filename
            if source_file.exists():
                shutil.copy(source_file, destination_file)
            else:
                print(f"Avertissement : Fichier DICOM non trouvé : {source_file}. Il sera ignoré.")

        # Lancer dcm2niix pour convertir la série DICOM en NIfTI
        try:
            # -z y: Compresser le fichier de sortie
            # -o : Répertoire de sortie
            # -f : Format du nom de fichier, ici "patient_{patient_id}"
            cmd = [
                "dcm2niix",
                "-z", "y",
                "-o", str(output_root),
                "-f", f"patient_{patient_id}",
                str(temp_patient_dir)
            ]
            
            subprocess.run(cmd, check=True, capture_output=True, text=True)
            print(f"  -> Conversion réussie pour {patient_id}. Fichier sauvegardé : patient_{patient_id}.nii.gz")

        except subprocess.CalledProcessError as e:
            print(f"  !! Erreur lors de la conversion de {patient_id}: {e.stderr}")
        except FileNotFoundError:
            print("Erreur : La commande 'dcm2niix' n'est pas installé dans l'environnement actuel.")
            
        # Supprimer le répertoire temporaire pour le nettoyage
        shutil.rmtree(temp_patient_dir)
    
    print(">>> Reconstruction des scans 3D terminée.")

# ======================
# EXÉCUTION DU SCRIPT
# ======================
if __name__ == "__main__":
    reconstruct_3d_scans_from_dicom(csv_path, dicom_dir, output_nifti_dir)

>>> Démarrage de la reconstruction des scans 3D...
Traitement du patient : ID_001fdc72 avec 36 images...
Erreur : La commande 'dcm2niix' n'est pas installé dans l'environnement actuel.
Traitement du patient : ID_00307f7a avec 28 images...
Erreur : La commande 'dcm2niix' n'est pas installé dans l'environnement actuel.
Traitement du patient : ID_00526c11 avec 52 images...
Erreur : La commande 'dcm2niix' n'est pas installé dans l'environnement actuel.
Traitement du patient : ID_005515a3 avec 32 images...
Erreur : La commande 'dcm2niix' n'est pas installé dans l'environnement actuel.
Traitement du patient : ID_0086733c avec 28 images...
Erreur : La commande 'dcm2niix' n'est pas installé dans l'environnement actuel.
Traitement du patient : ID_009882cb avec 40 images...
Erreur : La commande 'dcm2niix' n'est pas installé dans l'environnement actuel.
Traitement du patient : ID_009cae85 avec 35 images...


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import pydicom
import numpy as np
import nibabel as nib
from pathlib import Path
import os
import shutil
import subprocess
from collections import defaultdict
import logging

# Configuration du logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def load_dicom_metadata(dicom_path):
    """Charge les métadonnées DICOM nécessaires pour le tri"""
    try:
        ds = pydicom.dcmread(dicom_path, stop_before_pixels=True)
        return {
            'SliceLocation': getattr(ds, 'SliceLocation', None),
            'ImagePositionPatient': getattr(ds, 'ImagePositionPatient', None),
            'InstanceNumber': getattr(ds, 'InstanceNumber', None),
            'SliceThickness': getattr(ds, 'SliceThickness', None),
            'PixelSpacing': getattr(ds, 'PixelSpacing', None)
        }
    except Exception as e:
        logger.error(f"Erreur lecture métadonnées {dicom_path}: {e}")
        return None

def group_dicoms_by_patient(csv_path, dicom_dir):
    """Groupe les fichiers DICOM par PatientID"""
    df = pd.read_csv(csv_path)
    patient_groups = defaultdict(list)
    
    for _, row in df.iterrows():
        patient_id = row['PatientID']
        filename = row['filename']
        dicom_path = dicom_dir / filename
        
        if dicom_path.exists():
            patient_groups[patient_id].append({
                'filename': filename,
                'path': dicom_path,
                'labels': {col: row[col] for col in ['any', 'epidural', 'intraparenchymal', 
                                                   'intraventricular', 'subarachnoid', 'subdural']}
            })
        else:
            logger.warning(f"Fichier DICOM introuvable: {dicom_path}")
    
    return dict(patient_groups)

def sort_dicom_slices(dicom_files):
    """Trie les coupes DICOM selon leur position spatiale"""
    slices_info = []
    
    for dicom_info in dicom_files:
        metadata = load_dicom_metadata(dicom_info['path'])
        if metadata:
            dicom_info['metadata'] = metadata
            slices_info.append(dicom_info)
    
    # Tri par SliceLocation, puis ImagePositionPatient[2] (coordonnée Z), puis InstanceNumber
    def sort_key(item):
        meta = item['metadata']
        slice_loc = meta['SliceLocation'] if meta['SliceLocation'] is not None else 0
        
        if meta['ImagePositionPatient'] is not None:
            z_pos = float(meta['ImagePositionPatient'][2])
        else:
            z_pos = slice_loc
            
        instance_num = meta['InstanceNumber'] if meta['InstanceNumber'] is not None else 0
        
        return (z_pos, slice_loc, instance_num)
    
    return sorted(slices_info, key=sort_key)

def reconstruct_3d_volume_pydicom(sorted_slices, output_path):
    """Reconstruit un volume 3D à partir des coupes DICOM triées"""
    try:
        # Charger toutes les images
        slices = []
        for slice_info in sorted_slices:
            ds = pydicom.dcmread(slice_info['path'])
            slices.append(ds)
        
        # Vérifier la cohérence des dimensions
        ref_shape = slices[0].pixel_array.shape
        for i, slice_ds in enumerate(slices[1:], 1):
            if slice_ds.pixel_array.shape != ref_shape:
                logger.warning(f"Dimension incohérente pour la coupe {i}: {slice_ds.pixel_array.shape} vs {ref_shape}")
        
        # Construire le volume 3D
        volume = np.stack([s.pixel_array for s in slices], axis=-1)
        
        # Obtenir les informations d'espacement
        pixel_spacing = slices[0].PixelSpacing if hasattr(slices[0], 'PixelSpacing') else [1.0, 1.0]
        slice_thickness = slices[0].SliceThickness if hasattr(slices[0], 'SliceThickness') else 1.0
        
        # Créer l'affine matrix pour NIfTI (orientation standard)
        affine = np.diag([float(pixel_spacing[0]), float(pixel_spacing[1]), float(slice_thickness), 1.0])
        
        # Créer et sauvegarder l'image NIfTI
        nifti_img = nib.Nifti1Image(volume, affine)
        nib.save(nifti_img, output_path)
        
        logger.info(f"Volume 3D sauvegardé: {output_path} (shape: {volume.shape})")
        return True
        
    except Exception as e:
        logger.error(f"Erreur reconstruction volume: {e}")
        return False

def reconstruct_3d_volume_dcm2niix(sorted_slices, output_path, temp_dir):
    """Alternative utilisant dcm2niix pour la reconstruction"""
    try:
        # Créer un dossier temporaire pour ce patient
        patient_temp_dir = temp_dir / f"temp_{output_path.stem}"
        patient_temp_dir.mkdir(parents=True, exist_ok=True)
        
        # Copier les fichiers DICOM dans l'ordre
        for i, slice_info in enumerate(sorted_slices):
            temp_file = patient_temp_dir / f"{i:04d}_{slice_info['filename']}"
            shutil.copy2(slice_info['path'], temp_file)
        
        # Utiliser dcm2niix
        cmd = [
            "dcm2niix",
            "-z", "y",  # Compression gzip
            "-f", output_path.stem,  # Nom de fichier de sortie
            "-o", str(output_path.parent),  # Dossier de sortie
            str(patient_temp_dir)  # Dossier d'entrée
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode == 0:
            logger.info(f"dcm2niix réussi pour {output_path}")
            success = True
        else:
            logger.error(f"dcm2niix échoué: {result.stderr}")
            success = False
        
        # Nettoyer le dossier temporaire
        shutil.rmtree(patient_temp_dir)
        return success
        
    except Exception as e:
        logger.error(f"Erreur avec dcm2niix: {e}")
        return False

def save_patient_labels(patient_slices, output_dir, patient_id):
    """Sauvegarde les labels par patient"""
    labels_summary = defaultdict(int)
    slice_labels = []
    
    for slice_info in patient_slices:
        slice_labels.append({
            'filename': slice_info['filename'],
            **slice_info['labels']
        })
        
        # Compter les labels positifs
        for label_type, value in slice_info['labels'].items():
            if value == 1:
                labels_summary[label_type] += 1
    
    # Sauvegarder les informations de labels
    labels_file = output_dir / f"{patient_id}_labels.json"
    import json
    with open(labels_file, 'w') as f:
        json.dump({
            'patient_id': patient_id,
            'total_slices': len(slice_labels),
            'labels_summary': dict(labels_summary),
            'slice_details': slice_labels
        }, f, indent=2)

def reconstruct_all_patients(csv_path, dicom_dir, output_dir, cache_dir, method='pydicom'):
    """
    Fonction principale pour reconstruire tous les volumes 3D
    
    Args:
        csv_path: Chemin vers le fichier CSV
        dicom_dir: Dossier contenant les fichiers DICOM
        output_dir: Dossier de sortie pour les volumes NIfTI
        cache_dir: Dossier de cache
        method: 'pydicom' ou 'dcm2niix'
    """
    
    # Créer les dossiers de sortie
    output_dir = Path(output_dir)
    cache_dir = Path(cache_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    cache_dir.mkdir(parents=True, exist_ok=True)
    
    # Grouper les DICOM par patient
    logger.info("Groupement des DICOM par patient...")
    patient_groups = group_dicoms_by_patient(csv_path, dicom_dir)
    logger.info(f"Trouvé {len(patient_groups)} patients")
    
    successful_reconstructions = 0
    
    for patient_id, dicom_files in patient_groups.items():
        logger.info(f"Traitement du patient {patient_id} ({len(dicom_files)} coupes)")
        
        # Trier les coupes
        sorted_slices = sort_dicom_slices(dicom_files)
        if len(sorted_slices) == 0:
            logger.warning(f"Aucune coupe valide pour le patient {patient_id}")
            continue
        
        # Chemin de sortie
        output_path = output_dir / f"{patient_id}.nii.gz"
        
        # Reconstruction 3D
        if method == 'dcm2niix':
            success = reconstruct_3d_volume_dcm2niix(sorted_slices, output_path, cache_dir)
        else:
            success = reconstruct_3d_volume_pydicom(sorted_slices, output_path)
        
        if success:
            successful_reconstructions += 1
            # Sauvegarder les labels
            save_patient_labels(sorted_slices, output_dir, patient_id)
        
        logger.info(f"Patient {patient_id}: {'✓' if success else '✗'}")
    
    logger.info(f"Reconstruction terminée: {successful_reconstructions}/{len(patient_groups)} patients réussis")

# Exemple d'utilisation
if __name__ == "__main__":
    csv_path = Path("/home/tibia/Projet_Hemorragie/Seg_hemorragie/Classification_RSNA/data/csv/train_fold0.csv")
    dicom_dir = Path("/home/tibia/Projet_Hemorragie/Seg_hemorragie/Classification_RSNA/data/rsna-intracranial-hemorrhage-detection/stage_2_train")
    cache_dir = Path("./persistent_cache/fold0")
    output_dir = Path("./reconstructed_volumes/fold0")
    
    # Choisir la méthode: 'pydicom' ou 'dcm2niix'
    reconstruct_all_patients(
        csv_path=csv_path,
        dicom_dir=dicom_dir,
        output_dir=output_dir,
        cache_dir=cache_dir,
        method='pydicom'  # ou 'dcm2niix' si vous préférez
    )